### SuperTrend Date

In [2]:
coin = 'JASMY/USD'

import ccxt
import schedule
import time
import pandas as pd
import numpy as np
import json
from datetime import datetime

pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

In [1]:
# ------------------------------------------------------------------ #

def get_keys(path):    
    with open(path) as f:
        return json.load(f)

# ------------------------------------------------------------------ #

CB_API_KEY = get_keys('/Users/kyledufrane/.secret/cbpro_key.json')
CB_API_SECRET = get_keys('/Users/kyledufrane/.secret/cbpro_secret.json')
CB_API_PASSPHRASE = get_keys('/Users/kyledufrane/.secret/cbpro_passphrase.json')

exchange = ccxt.coinbasepro({'apiKey': CB_API_KEY['key'],
                             'secret': CB_API_SECRET['secret'],
                             'password': CB_API_PASSPHRASE['passphrase']})

# bars = exchange.fetch_ohlcv(coin, timeframe='1m', limit=100)

# df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# ------------------------------------------------------------------ #

def true_range(dataframe):
    dataframe['previous_close'] = dataframe['close'].shift(1)
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-previous_close'] = abs(dataframe['high'] - dataframe['previous_close'])
    dataframe['low-previous_close'] = abs(dataframe['low'] - dataframe['previous_close'])
    true_range = dataframe[['high-low', 'high-previous_close', 'low-previous_close']].max(axis=1)  
    return true_range

# ------------------------------------------------------------------ #


def atr(dataframe, period=5):   
    dataframe['true_range'] = true_range(dataframe)   
#     print("Calculating average true range")   
    the_atr = dataframe['true_range'].rolling(period).mean()   
    return the_atr

# ------------------------------------------------------------------ #


def super_trend(dataframe, period=7, multiplier=3):
    hl2 = (dataframe['high'] + dataframe['low']) / 2
    dataframe['atr'] = atr(dataframe, period=period)
    dataframe['upperband'] = hl2 + (multiplier * dataframe['atr'])
    dataframe['lowerband'] = hl2 - (multiplier * dataframe['atr'])
    dataframe['in_uptrend'] = True
    
    for current in range(1, len(dataframe.index)):
        previous = current - 1
        
        if dataframe['close'][current] > dataframe['upperband'][previous]:
            dataframe['in_uptrend'][current]= True
        elif dataframe['close'][current] < dataframe['lowerband'][previous]:
            dataframe['in_uptrend'][current] = False
        else:
            dataframe['in_uptrend'][current] = dataframe['in_uptrend'][previous]
            if dataframe['in_uptrend'][current] and dataframe['lowerband'][current] < dataframe['lowerband'][previous]:
                dataframe['lowerband'][current] = dataframe['lowerband'][previous]
            if not dataframe['in_uptrend'][current] and dataframe['upperband'][current] > dataframe['upperband'][previous]:
                dataframe['upperband'][current] = dataframe['upperband'][previous]
    return dataframe

# ------------------------------------------------------------------ #

in_position = False

# ------------------------------------------------------------------ #

def check_buy_sell_signal(dataframe):
    global in_position
    last_row = len(dataframe.index) - 1
    previous_row = last_row - 1
        
    if dataframe['in_uptrend'][last_row] != dataframe['in_uptrend'][previous_row]:
        print('****************** CHANGE IN STATE ******************')
        
    if not dataframe['in_uptrend'][previous_row] and dataframe['in_uptrend'][last_row]:
        print('Changing to uptrend, buy')
        if not in_position:
            order = exchange.create_market_buy_order(coin, 10)
            print(order)
            in_position=True
        else: 
            print("Already in position, nothing to do")
    
    if dataframe['in_uptrend'][previous_row] and not dataframe['in_uptrend'][last_row]:
        print('Changing to uptrend, sell')
        if in_position:
            order = exchange.create_limit_sell_order(coin, 10)
            print(order)
            in_position=False
        else:
            print("You aren't in position, nothing to sell")
    return None

# ------------------------------------------------------------------ #            
            
def run_bot():
#     print(f"Fetching new bars for {datetime.now().isoformat()}....")
    bars = exchange.fetch_ohlcv(coin, timeframe='1m', limit=100)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    supertrend_data = super_trend(df)
    check_buy_sell_signal(supertrend_data)

schedule.every(4).seconds.do(run_bot)

while(True):
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [3]:

CB_API_KEY = get_keys('/Users/kyledufrane/.secret/cbpro_key.json')
CB_API_SECRET = get_keys('/Users/kyledufrane/.secret/cbpro_secret.json')
CB_API_PASSPHRASE = get_keys('/Users/kyledufrane/.secret/cbpro_passphrase.json')

exchange = ccxt.coinbasepro({'apiKey': CB_API_KEY['key'],
                             'secret': CB_API_SECRET['secret'],
                             'password': CB_API_PASSPHRASE['passphrase']})

bars = exchange.fetch_ohlcv(coin, timeframe='1m', limit=150)

df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

NameError: name 'get_keys' is not defined

In [11]:
exchange.timeframes

{'1m': 60, '5m': 300, '15m': 900, '1h': 3600, '6h': 21600, '1d': 86400}

In [4]:
import pandas as pd

In [5]:
len(pd.read_csv('data/JASMY-USD.csv', error_bad_lines=False))

99

In [15]:
df = pd.read_csv('data/JASMY-USD.csv')

In [16]:
df.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,volume,previous_close,high-low,high-previous_close,low-previous_close,true_range,atr,upperband,lowerband,in_uptrend
3163,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,217072.0,0.23002,0.00116,0.00047,0.00069,0.00116,0.002821,0.238374,0.221446,False
3164,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,293894.0,0.23049,0.00157,0.00055,0.00102,0.00157,0.002801,0.238374,0.221851,False
3165,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,907016.0,0.23071,0.00329,0.00329,0.00000,0.00329,0.002561,0.238374,0.224671,False
3166,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,246169.0,0.23398,0.00236,0.00032,0.00204,0.00236,0.002214,0.238374,0.226477,False
3167,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123423.0,0.23409,0.00109,0.00000,0.00109,0.00109,0.002051,0.238374,0.227391,False


In [9]:
coin = 'JASMY/USD'


def get_keys(path):    
    with open(path) as f:
        return json.load(f)

# ------------------------------------------------------------------ #

CB_API_KEY = get_keys('/Users/kyledufrane/.secret/cbpro_key.json')
CB_API_SECRET = get_keys('/Users/kyledufrane/.secret/cbpro_secret.json')
CB_API_PASSPHRASE = get_keys('/Users/kyledufrane/.secret/cbpro_passphrase.json')

exchange = ccxt.coinbasepro({'apiKey': CB_API_KEY['key'],
                             'secret': CB_API_SECRET['secret'],
                             'password': CB_API_PASSPHRASE['passphrase']})

bars = exchange.fetch_ohlcv(coin, timeframe='1m', limit=100)

df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


def true_range(dataframe):
    dataframe['previous_close'] = dataframe['close'].shift(1)
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-previous_close'] = abs(dataframe['high'] - dataframe['previous_close'])
    dataframe['low-previous_close'] = abs(dataframe['low'] - dataframe['previous_close'])
    true_range = dataframe[['high-low', 'high-previous_close', 'low-previous_close']].max(axis=1)  
    return true_range

# ------------------------------------------------------------------ #


def atr(dataframe, period=5):   
    dataframe['true_range'] = true_range(dataframe)   
#     print("Calculating average true range")   
    the_atr = dataframe['true_range'].rolling(period).mean()   
    return the_atr

# ------------------------------------------------------------------ #


def super_trend(dataframe, period=7, multiplier=3):
    hl2 = (dataframe['high'] + dataframe['low']) / 2
    dataframe['atr'] = atr(dataframe, period=period)
    dataframe['upperband'] = hl2 + (multiplier * dataframe['atr'])
    dataframe['lowerband'] = hl2 - (multiplier * dataframe['atr'])
    dataframe['in_uptrend'] = True
    
    for current in range(1, len(dataframe.index)):
        previous = current - 1
        
        if dataframe['close'][current] > dataframe['upperband'][previous]:
            dataframe['in_uptrend'][current]= True
        elif dataframe['close'][current] < dataframe['lowerband'][previous]:
            dataframe['in_uptrend'][current] = False
        else:
            dataframe['in_uptrend'][current] = dataframe['in_uptrend'][previous]
            if dataframe['in_uptrend'][current] and dataframe['lowerband'][current] < dataframe['lowerband'][previous]:
                dataframe['lowerband'][current] = dataframe['lowerband'][previous]
            if not dataframe['in_uptrend'][current] and dataframe['upperband'][current] > dataframe['upperband'][previous]:
                dataframe['upperband'][current] = dataframe['upperband'][previous]
    return dataframe

In [11]:
x = super_trend(df)

In [18]:
df = pd.read_csv('data/JASMY-USD.csv')

EmptyDataError: No columns to parse from file

In [14]:
df

,timestamp,open,high,low,close,volume,previous_close,high-low,high-previous_close,low-previous_close,true_range,atr,upperband,lowerband,in_uptrend
0,2021-11-05 00:57:00,0.23460,0.24113,0.23352,0.24043,1604992.0,NaN,0.00761,NaN,NaN,0.00761,NaN,NaN,NaN,True
1,2021-11-05 00:58:00,0.24005,0.24200,0.23506,0.24121,2047955.0,0.24043,0.00694,0.00157,0.00537,0.00694,NaN,NaN,NaN,True
2,2021-11-05 00:59:00,0.24158,0.24400,0.23980,0.24036,2554277.0,0.24121,0.00420,0.00279,0.00141,0.00420,NaN,NaN,NaN,True
3,2021-11-05 01:00:00,0.24062,0.24100,0.23000,0.23554,2968889.0,0.24036,0.01100,0.00064,0.01036,0.01100,NaN,NaN,NaN,True
4,2021-11-05 01:01:00,0.23554,0.23853,0.23504,0.23639,1103498.0,0.23554,0.00349,0.00299,0.00050,0.00349,NaN,NaN,NaN,True
5,2021-11-05 01:02:00,0.23685,0.23800,0.23530,0.23697,502591.0,0.23639,0.00270,0.00161,0.00109,0.00270,NaN,NaN,NaN,True
6,2021-11-05 01:03:00,0.23695,0.23989,0.23627,0.23859,901602.0,0.23697,0.00362,0.00292,0.00070,0.00362,0.005651,0.255034,0.221126,True
7,2021-11-05 01:04:00,0.23773,0.24082,0.23773,0.24080,392629.0,0.23859,0.00309,0.00223,0.00086,0.00309,0.005006,0.254292,0.224258,True
8,2021-11-05 01:05:00,0.24058,0.24400,0.24017,0.24238,1326835.0,0.24080,0.00383,0.00320,0.00063,0.00383,0.004561,0.255769,0.228401,True
9,2021-11-05 01:06:00,0.24285,0.24540,0.24160,0.24431,1659973.0,0.24238,0.00380,0.00302,0.00078,0.00380,0.004504,0.257013,0.229987,True
